# Hydrodynamical modelling
Here I create a hydrodynamical modelling workflow using a fork of [Delft3D Toolbox](https://github.com/VeckoTheGecko/Delft3D-Toolbox).

The goal with this notebook is to establish a working pipeline from the top bullet point to the bottom:
- Exact resolution PNG
- Delft3D Meshgrid
- Delft3D Simulation file
- Run the simulation
- NetCDF output


Key assumption:
The boundary files `simulation.bct` and `simulation.bnd` fundamentally rely on the domain being (w, h) = (120, 120)


### PNG read

In [1]:
import delft3d
import numpy as np
import xarray as xr
import pandas as pd
from typing import Iterable
import typing
import os
import time
from matplotlib.image import imread
import matplotlib.pyplot as plt
from delft3d import Simulation

from pathlib import Path

from hydro_mesh import HydroMesh
import toml
from env import HYDRO_FOLDER, PARTICLE_SIM_FOLDER, CONFIG_FOLDER, TEMPLATE_FOLDER
import shutil

from __init__ import logger

In [2]:
cfg = CONFIG_FOLDER / "headland2_1km_oninkb.toml"
cfg = toml.load(cfg)


In [3]:
# Defining folder that hydro simulation is going in
hydro_sim_run = HYDRO_FOLDER / cfg["hydro_model"]["ID_human"]

if (hydro_sim_run / "trim-simulation.nc").exists():
    logger.warning(f"Hydrodynamic simulation run already exists at {hydro_sim_run}.")
else:
    if hydro_sim_run.exists():
        shutil.rmtree(hydro_sim_run)
    resolution = int(cfg["coastline"]["dx"] / 1_000)
    shutil.copytree(TEMPLATE_FOLDER / f"hydro_template_{resolution}km", hydro_sim_run)

    hydro = HydroMesh(
        x_start=cfg["coastline"]["start_xy"][0],
        y_start=cfg["coastline"]["start_xy"][1],
        dx = cfg["coastline"]["dx"],
        dy = cfg["coastline"]["dy"],
        )


    coast_path = cfg["coastline"]["coast_file"]
    hydro.from_png(coast_path)
    hydro.to_grd(template_grd = TEMPLATE_FOLDER / "hydro_example" / "simulation.grd", out_grd = hydro_sim_run / "simulation.grd", missing_value=0.0)
    hydro.to_netcdf(path = hydro_sim_run / "land_mask.nc")

    # Prompting user to manually generate the enclosure file
    counter = 0
    while True:
        if (hydro_sim_run / "simulation.enc").exists():
            break
        elif counter < 2:
            input(f"Please open the simulation.grd file in folder '{hydro_sim_run}' in Delft3D and save it (generating simulation.enc). Press enter when done.")
            counter += 1
        else:
            raise FileNotFoundError("Could not find simulation.enc. Please open the simulation.grd file in Delft3D and save it (generating simulation.enc).")

    # Run the simulation manually
    logger.success(f"Simulation files ready for sim: {cfg['hydro_model']['ID_human']}")
    logger.info(f"Please run the simulation manually: {hydro_sim_run.resolve()}")

{'Coordinate System': 'Cartesian', 'Missing Value': 0.0, 'MN': [121, 121]}


2022-09-22 00:22:35.059 | SUCCESS  | __main__:<module>:40 - Simulation files ready for sim: yellow-delaware-avocado
2022-09-22 00:22:35.065 | INFO     | __main__:<module>:41 - Please run the simulation manually: /mnt/c/Users/nickh/oceans/repos/honours-modelling/my_modelling/sim-runs-hydro/yellow-delaware-avocado
